## Natural Language to SQL Proof of Concept

A simple proof of concept that allows a non-programmer to write a program in natural language and have it converted to SQL Code.

#### What is natural language?
A natural language, as described when [Dictionary.com](https://www.dictionary.com/browse/natural-language) states, "a language that has developed and evolved naturally, through use by human beings, as opposed to an invented or constructed language, as a computer programming language". Simply put, a natural language is a language that is used by humans to communicate with each other. The setences that you are reading right now are written in natural language.

#### What is SQL?
SQL, or Structured Query Language, is a "Structured Query Language (SQL) is a standardized programming language that is used to manage relational databases and perform various operations on the data in them" [Techtarget.com](https://www.techtarget.com/searchdatamanagement/definition/SQL#:~:text=Structured%20Query%20Language%20(SQL)%20is,on%20the%20data%20in%20them.).

#### The difference between natural language and SQL
Natural language is used to communicate with other humans. SQL is used to communicate with a database. The two languages are very different. For example, in natural language, you can say:
    
    "I want to buy a car, but my budget is no more than $10000"

In SQL, you would say:

    "SELECT * FROM cars WHERE price < 10000"
    
The two sentences are very different, but they both mean the same thing. An engineer who understands both the natural language and SQL can easily convert the two back and forward. If a non-technical tells them they need a report for all sales within the last 30 days, the engineer can easily convert that to SQL and get the data. 

What we want to do is show that not only can an engineer do it, but GPT can as well.


#### The Imports

We will need a number of different python libraries to make this work. We will need to import the following libraries:

In [2]:
# Installed Package Imports
import pandas as pd
import openai
import dotenv
import sqlalchemy

In [3]:
# Python Imports
import os

#### Safeguarding our API Key

We can easily obfuscate the API Key that we are using by storing it in a environment variables file, or a .env file. We can then import the .env file into our notebook and use the API Key without having to worry about it being exposed.

The .env will have an entry that looks like this:

OPENAI=Our-API-Key-is-Stored-Here

This is imported using a package called Dotenv. 

In [4]:
# API Key is Stored in a .env File, please see the .env.example file for more information
dotenv.load_dotenv()
openai.api_key = os.getenv("OPENAPI")

#### Data Exploration

First we must import and explore our data just to get an idea of what we are working with

In [5]:
# Read in the sample data 
df = pd.read_csv("data/sales_data_sample.csv")

In [6]:
# View the first 5 rows of the data
df.head()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,Shipped,1,2,2003,...,897 Long Airport Avenue,NaN,NYC,NY,10022,USA,NaN,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,5/7/2003 0:00,Shipped,2,5,2003,...,59 rue de l'Abbaye,NaN,Reims,NaN,51100,France,EMEA,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,Shipped,3,7,2003,...,27 rue du Colonel Pierre Avia,NaN,Paris,NaN,75508,France,EMEA,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,Shipped,3,8,2003,...,78934 Hillside Dr.,NaN,Pasadena,CA,90003,USA,NaN,Young,Julie,Medium
4,10159,49,100.00,14,5205.27,10/10/2003 0:00,Shipped,4,10,2003,...,7734 Strong St.,NaN,San Francisco,CA,NaN,USA,NaN,Brown,Julie,Medium


#### Database Setup

As we want to perform SQL queries we need to store our CSV data in a database. We can do this using SQL Alchemy, using the module called create_engine. We will create a database called "temp_db" that is stored in memory. We then push our data into the database in a table called "sales".

After that, we can perform queries against the database using the text module that we import from SQL Alchemy.

In [7]:
from sqlalchemy import create_engine
from sqlalchemy import text 

In [8]:
# First we are going to create a temporary database in memory
temp_db = create_engine('sqlite:///:memory:', echo=True)

In [9]:
# Now we push the Pandas DF to the database that we can perform SQL queries on. As we set echo to true, we get to see the SQL query that is being executed while creating the table
data = df.to_sql('sales', con=temp_db, if_exists='replace')

2023-03-09 10:27:29,486 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("sales")
2023-03-09 10:27:29,487 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-09 10:27:29,488 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("sales")
2023-03-09 10:27:29,489 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-09 10:27:29,492 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-09 10:27:29,493 INFO sqlalchemy.engine.Engine 
CREATE TABLE sales (
	"index" BIGINT, 
	"ORDERNUMBER" BIGINT, 
	"QUANTITYORDERED" BIGINT, 
	"PRICEEACH" FLOAT, 
	"ORDERLINENUMBER" BIGINT, 
	"SALES" FLOAT, 
	"ORDERDATE" TEXT, 
	"STATUS" TEXT, 
	"QTR_ID" BIGINT, 
	"MONTH_ID" BIGINT, 
	"YEAR_ID" BIGINT, 
	"PRODUCTLINE" TEXT, 
	"MSRP" BIGINT, 
	"PRODUCTCODE" TEXT, 
	"CUSTOMERNAME" TEXT, 
	"PHONE" TEXT, 
	"ADDRESSLINE1" TEXT, 
	"ADDRESSLINE2" TEXT, 
	"CITY" TEXT, 
	"STATE" TEXT, 
	"POSTALCODE" TEXT, 
	"COUNTRY" TEXT, 
	"TERRITORY" TEXT, 
	"CONTACTLASTNAME" TEXT, 
	"CONTACTFIRSTNAME" TEXT, 
	"DEALSIZE" TEXT
)


2023

In [10]:
# Now we can perform SQL queries on the data
with temp_db.connect() as con:
    rs = con.execute(text("SELECT * FROM sales LIMIT 5"))
    for row in rs:
        print(row)

2023-03-09 10:27:30,034 INFO sqlalchemy.engine.Engine SELECT * FROM sales LIMIT 5
2023-03-09 10:27:30,035 INFO sqlalchemy.engine.Engine [generated in 0.00078s] ()
(0, 10107, 30, 95.7, 2, 2871.0, '2/24/2003 0:00', 'Shipped', 1, 2, 2003, 'Motorcycles', 95, 'S10_1678', 'Land of Toys Inc.', '2125557818', '897 Long Airport Avenue', None, 'NYC', 'NY', '10022', 'USA', None, 'Yu', 'Kwai', 'Small')
(1, 10121, 34, 81.35, 5, 2765.9, '5/7/2003 0:00', 'Shipped', 2, 5, 2003, 'Motorcycles', 95, 'S10_1678', 'Reims Collectables', '26.47.1555', "59 rue de l'Abbaye", None, 'Reims', None, '51100', 'France', 'EMEA', 'Henriot', 'Paul', 'Small')
(2, 10134, 41, 94.74, 2, 3884.34, '7/1/2003 0:00', 'Shipped', 3, 7, 2003, 'Motorcycles', 95, 'S10_1678', 'Lyon Souveniers', '+33 1 46 62 7555', '27 rue du Colonel Pierre Avia', None, 'Paris', None, '75508', 'France', 'EMEA', 'Da Cunha', 'Daniel', 'Medium')
(3, 10145, 45, 83.26, 6, 3746.7, '8/25/2003 0:00', 'Shipped', 3, 8, 2003, 'Motorcycles', 95, 'S10_1678', 'Toys4G

#### NLP to SQL

To do this we are going to create a couple of functions:

1. A function that creates a table structure defintion from the DataFrame
2. A function that grabs the user's language input.
3. A function that combines prmops for our API call

#### Create Table Definition Function

In [11]:
def create_table_def(df):
    prompt = """### Sqlite SQL table, with it's properties:
    #
    # Sales({})
    #
    """.format(",".join(str(col) for col in df.columns))

    return prompt

A breakdown of the Create Table Def Function:

```
    Sales({})
```
We create a table called sales, with the columns to be injected where we placed the {}

```
    .format(",".join(str(col) for col in df.columns))
```

We then loop through the columns in our DataFrame and add them to the table definition. We also add a comma between each column.

As such we end up with the following:

```
    Sales(col1, col2, col3, col4, ...)
```

#### Get User Input Function

In [14]:
def prompt():
    user_prompt = input("Please enter your prompt: ")
    return user_prompt

A breakdown of the User Input :

This is a simple function that just promppts the user for an input, which is then returned.

An example of this:

```
    prompt()
```

```
    $ Please enter your prompt: "I want to buy a car, but my budget is no more than $10000"
```

What is returned as a variable:
```
    "I want to buy a car, but my budget is no more than $10000"
```


### Combination of the Prompts

In [22]:
def combine_prompts(df, user_prompt):
    definition = create_table_def(df)
    query_string = f"### A query to answer: {user_prompt}\nSELECT"
    return definition + query_string

### Breakdown of the Combination Function

This function is passed two parameters, a Pandas DataFrame and a User prompt string.

First it creats a table definition using the Create Table Def Function and the passed DataFrame.

```
    definition = create_table_def(df)
```

Then it creates a query string, ending in the start of the SQL query, with the passed User Prompt.

```
    query_string = f"### A query to answer: {user_prompt}\nSELECT"
```

Finally, it combines the two prompts and returns them.

In [23]:
# First we grab the natural language
nlp_text = prompt()

# Then we pass the data frame and the prompt to the function
combine_prompts(df, nlp_text)

"### Sqlite SQL table, with it's properties:\n    #\n    # Sales(ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,PHONE,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE)\n    #\n    ### A query to answer: I want the total sales by year\nSELECT"

This is what is return after we run the above cell with the user input "Grab all sales per month". This is what we will send to GPT.

        "### Sqlite SQL table, with it's properties:\n
        #\n
        # Sales(ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,
        YEAR_ID,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,PHONE,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,
        COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE)\n
        #\n    ### A query to answer: I want the total sales by year\nSELECT"

The final SELECT is not actually needed, but as GPT can be thought of partially as a text completition engine, it will push it towards creating an SQL query.

#### GPT API Call

Now we can pass our prompt to the GPT API. To do this, we will call the OpenAI Completion Create module that requires a few parameters:

* Engine: This is the engine that we want to use. We will use the DAVINCI engine, first we will use the code specific 'code-davinci-002' and then later we will try the updated 'text-davinci-003'

* Prompt: This is the prompt that we want to pass to GPT. This is the combination of the table definition and the user prompt.

* Temperature: This is the temperature of the model. The higher the temperature, the more likely the model is to take a risk and generate novel, creative text. Lower temperatures result in more likely text. We will use a temperature of 0 as we do not want a random response and we are looking for something very specific

* Max Tokens: This is the maximum number of tokens that the model will return. This needs to be large enough to get a full SQL Query back, but not too large that it starts to return gibberish. We will use 200.

* Top P: This is the cumulative probability that will be used to truncate the distribution. We will use 1 as we want to get the most likely response.

* Frequency Penalty: This is a parameter that penalizes new tokens based on their existing frequency in the text so far. This is used to encourage diversity in the response. We will use 0 as we want to get the most likely response.

* Presence Penalty: This is a parameter that penalizes new tokens based on whether they appear in the text so far. This is used to encourage diversity in the response. We will use 0 as we want to get the most likely response.

* Stop Sequence: This is a string that tells the model to stop generating text when it encounters the string. We will use both a '#' to stop if it starts commenting as this would mean it has finished the SQL Query, as well as a ';' as this is the end of an SQL Query.

In [24]:
response = openai.Completion.create(
    engine = "code-davinci-002",
    prompt = combine_prompts(df, nlp_text),
    temperature = 0,
    max_tokens = 200,
    top_p = 1,
    frequency_penalty = 0,
    presence_penalty = 0,
    stop = ['#', ';']
)

In [25]:
response['choices'][0]

<OpenAIObject at 0x2b111c1fd30> JSON: {
  "finish_reason": "stop",
  "index": 0,
  "logprobs": null,
  "text": " YEAR_ID, SUM(SALES) AS TOTAL_SALES\nFROM Sales\nGROUP BY YEAR_ID\nORDER BY YEAR_ID"
}

#### Handing The Response

The response will need to be handled, as although we passed the SELECT to kick of an SQL Query, if you look above you can see that it was not inlcuded within the response.

To do this, we look for the white space at the begining of the response, and then add the SELECT to the begining of the response.

If there is no quite space, we check if the response starts with a SELECT, and if it does, we return the response.

Otherwise we add SELECT and a space to the start of the response.

In [31]:
def handle_response(response):
    query = response['choices'][0]['text']
    if query.startswith(" "):
        query = "SELECT" + query
    elif query.startswith("SELECT"):
        query = query
    else:
        query = "SELECT " + query
    return query

Now that we have written the response handler, we can pass our response to it and get our SQL Query

In [28]:
handle_response(response)

'SELECT YEAR_ID, SUM(SALES) AS TOTAL_SALES\nFROM Sales\nGROUP BY YEAR_ID\nORDER BY YEAR_ID'

Our SQL Query is:

    SELECT YEAR_ID, SUM(SALES) AS TOTAL_SALES\nFROM Sales\nGROUP BY YEAR_ID\nORDER BY YEAR_ID
   
We can now run it against our database and check the results

In [29]:

with temp_db.connect() as con:
    result = con.execute(text(handle_response(response)))

2023-03-09 10:49:18,064 INFO sqlalchemy.engine.Engine SELECT YEAR_ID, SUM(SALES) AS TOTAL_SALES
FROM Sales
GROUP BY YEAR_ID
ORDER BY YEAR_ID
2023-03-09 10:49:18,065 INFO sqlalchemy.engine.Engine [generated in 0.00113s] ()


In [30]:
result.all()

[(2003, 3516979.540000001), (2004, 4724162.599999997), (2005, 1791486.71)]

#### Results

We can see that the results are correct, and we have successfully created a NLP to SQL Query engine.

We asked for total sales by year, and we got the correct SQL Query to do this.

Our SQL Response was a Python list of tuples, with the year matched to it's corresponding sum of sales.